In [1]:
from datasets import load_dataset, load_from_disk
import os
from pathlib import Path
from typing import Dict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    DataCollatorForSeq2Seq,
    DataCollatorWithPadding,
    TrainingArguments,
    Seq2SeqTrainingArguments,
    Trainer,
    Seq2SeqTrainer,
    PreTrainedTokenizerFast,
)
from functools import partial
from transformers.models.qwen2.tokenization_qwen2_fast import Qwen2TokenizerFast
from transformers.models.qwen2.modeling_qwen2 import Qwen2ForCausalLM, Qwen2ForSequenceClassification
from peft import PrefixTuningConfig, get_peft_model, TaskType
from peft.peft_model import PeftModelForCausalLM, PeftModelForSequenceClassification
from rouge_score import rouge_scorer
import numpy as np
from transformers import GenerationConfig
import torch

In [2]:
torch.cuda.empty_cache()

In [3]:
cache_dir = '/root/autodl-tmp'
base_model_id = "tiansz/bert-base-chinese"
sft_model_path = "/root/llm_adv_qa/resources/sft_models/classification/best"
sft_model_path

('/root/autodl-tmp/.cache/modelscope/hub/tiansz/bert-base-chinese',
 '/root/llm_adv_qa/resources/sft_models/classification/best')

In [4]:
id2label={
    0: "A",
    1: "B",
    2: "C",
    3: "D",
    4: "E",
    5: "F"
}

label2id={
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5
}

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
type(tokenizer)

transformers.models.bert.tokenization_bert_fast.BertTokenizerFast

In [8]:
prompt_template = """
        请问“{question}”是属于下面哪个类别的问题?
        A: 公司基本信息,包含股票简称, 公司名称, 外文名称, 法定代表人, 注册地址, 办公地址, 公司网址网站, 电子信箱等.
        B: 公司员工信息,包含员工人数, 员工专业, 员工类别, 员工教育程度等.
        C: 财务报表相关内容, 包含资产负债表, 现金流量表, 利润表 中存在的字段, 包括费用, 资产，金额，收入等.
        D: 计算题,无法从年报中直接获得,需要根据计算公式获得, 包括增长率, 率, 比率, 比重,占比等. 
        E: 统计题，需要从题目获取检索条件，在数据集/数据库中进行检索、过滤、排序后获得结果.        
        F: 开放性问题,包括介绍情况,介绍方法,分析情况,分析影响,什么是XXX.
        你只需要回答字母编号, 不要回答字母编号及选项文本外的其他内容.
        """

questions = [
        "无形资产是什么?",
        "什么是归属于母公司所有者的综合收益总额？",
        "航发动力2019年的非流动负债比率保留两位小数是多少？",
        "在上海注册的所有上市公司中，2021年货币资金最高的前3家上市公司为？金额为？",
    ]

In [10]:
sft_model = AutoModelForSequenceClassification.from_pretrained(sft_model_path)

In [11]:
sft_model.classifier.state_dict()

OrderedDict([('weight',
              tensor([[ 0.0133, -0.0199, -0.0037,  ...,  0.0316, -0.0063, -0.0348],
                      [ 0.0063, -0.0316,  0.0029,  ..., -0.0237, -0.0049, -0.0034],
                      [ 0.0092, -0.0053,  0.0013,  ...,  0.0177, -0.0069,  0.0169],
                      [-0.0209,  0.0126,  0.0195,  ..., -0.0177,  0.0019,  0.0080],
                      [-0.0074, -0.0024, -0.0031,  ...,  0.0362, -0.0056, -0.0042],
                      [ 0.0189, -0.0230, -0.0157,  ..., -0.0100, -0.0052, -0.0078]])),
             ('bias',
              tensor([ 0.0015, -0.0004,  0.0007, -0.0005, -0.0019, -0.0005]))])

In [12]:
for q in questions:
    input_text = prompt_template.format(question=f"{q}")
    inputs = tokenizer(input_text, return_tensors="pt")
    with torch.no_grad():
        outputs = sft_model(**inputs)
    logits = outputs.logits
    print(logits)
    predicted_class = torch.argmax(logits, dim=-1).item()
    print(input_text)
    print(id2label[predicted_class])


tensor([[-0.1833, -1.8253,  1.4419, -1.5689, -1.8890,  2.1472]])

        请问“无形资产是什么?”是属于下面哪个类别的问题?
        A: 公司基本信息,包含股票简称, 公司名称, 外文名称, 法定代表人, 注册地址, 办公地址, 公司网址网站, 电子信箱等.
        B: 公司员工信息,包含员工人数, 员工专业, 员工类别, 员工教育程度等.
        C: 财务报表相关内容, 包含资产负债表, 现金流量表, 利润表 中存在的字段, 包括费用, 资产，金额，收入等.
        D: 计算题,无法从年报中直接获得,需要根据计算公式获得, 包括增长率, 率, 比率, 比重,占比等. 
        E: 统计题，需要从题目获取检索条件，在数据集/数据库中进行检索、过滤、排序后获得结果.        
        F: 开放性问题,包括介绍情况,介绍方法,分析情况,分析影响,什么是XXX.
        你只需要回答字母编号, 不要回答字母编号及选项文本外的其他内容.
        
F
tensor([[-1.5654, -2.1001,  2.4501, -2.3720, -0.8575,  2.2102]])

        请问“什么是归属于母公司所有者的综合收益总额？”是属于下面哪个类别的问题?
        A: 公司基本信息,包含股票简称, 公司名称, 外文名称, 法定代表人, 注册地址, 办公地址, 公司网址网站, 电子信箱等.
        B: 公司员工信息,包含员工人数, 员工专业, 员工类别, 员工教育程度等.
        C: 财务报表相关内容, 包含资产负债表, 现金流量表, 利润表 中存在的字段, 包括费用, 资产，金额，收入等.
        D: 计算题,无法从年报中直接获得,需要根据计算公式获得, 包括增长率, 率, 比率, 比重,占比等. 
        E: 统计题，需要从题目获取检索条件，在数据集/数据库中进行检索、过滤、排序后获得结果.        
        F: 开放性问题,包括介绍情况,介绍方法,分析情况,分析影响,什么是XXX.
        你只需要回答字母编号, 不要回答字母编号